# <u>S</u>tabilizer <u>S</u>ub<u>S</u>pace (S3) Projections

A library providing the necessary functionality to perform stabilizer subspace projections over systems of Pauli 
operators.

This facilitates implementations of qubit reduction techniques such as [*tapering*](https://arxiv.org/abs/1701.08213) and [*Contextual-Subspace VQE*](https://doi.org/10.22331/q-2021-05-14-456), both of which are demonstrated here.

In [1]:
from qreduce.tapering import tapering
from qreduce.cs_vqe import cs_vqe
import qreduce.utils.qonversion_tools as qonvert
from qreduce.utils.operator_toolkit import exact_gs_energy, plot_ground_state_amplitudes

import numpy as np
import openfermion as of
import openfermionpyscf as ofpyscf
from itertools import combinations
import json
import numpy as np

First of all, we construct our molecular Hamiltonian:

In [13]:
# Set molecule parameters

geometry = [('Be',(0,0,0))]

#geometry = [
#    ("O",(0,0,0)),
#    ("H",(0.952519,0,0)),
#    ("H",(-0.246530058,0.9200627021,0))
#    ]

# ...and one for which it does not.
#geometry=[
#    ('H', (0.0,0.0,0.0)),
#    ('H', (2.45366053071732,0.0,0.0)),
#    ('H', (2.45366053071732,2.45366053071732,0.0)),
#    ('H', (0.0,2.45366053071732,0.0))
#     ]  
    
basis = 'sto-3g'
multiplicity = 1
charge = 0

molecule_data = of.MolecularData(geometry, basis, multiplicity, charge)
#molecule.load()

# Run pyscf.
molecule = ofpyscf.run_pyscf(molecule_data,
                     run_scf=1,run_mp2=1,run_cisd=1,run_ccsd=1,run_fci=1)

n_qubits    = 2*molecule.n_orbitals
n_electrons = molecule.n_electrons

ham_fermionic = of.get_fermion_operator(molecule.get_molecular_hamiltonian())
ham_jw = of.jordan_wigner(ham_fermionic)

ham_dict = qonvert.QubitOperator_to_dict(ham_jw, n_qubits)

#print('Jordan-Wigner Hamiltonian:\n\n', ham_dict)
#print('Amplitude histogram for the ground state of the full system:')
amps, true_gs = plot_ground_state_amplitudes(ham_dict, n_qubits, return_amps=True)

Perform qubit tapering to reduce the number of necessary qubits:

In [6]:
molecule.hf_energy

-14.351880476202023

In [7]:
taper_hamiltonian = tapering(hamiltonian=ham_dict, 
                               ref_state=[int(i) for i in amps[0][0]])
ham_tap = taper_hamiltonian.taper_it()
tap_gs_energy, tap_gs = exact_gs_energy(ham_tap._dict)

print(f'In the sector {taper_hamiltonian.symmetry_sec}, we find the ground state energy to be {tap_gs_energy}.')
print(f'The absolute error is {tap_gs_energy-true_gs}.')

In the sector [1, 1, 1, 1, 1], we find the ground state energy to be -14.403655108067685.
The absolute error is 1.4210854715202004e-14.


Build CS-VQE model with legacy code for comparison:

In [8]:
from qreduce.utils.cs_vqe_tools_legacy import csvqe_approximations_heuristic, quasi_model, greedy_dfs

ham = ham_tap._dict
terms_noncon = greedy_dfs(ham, cutoff=5)[-1]
ham_noncon = {op:ham[op] for op in terms_noncon}
n_qubits = ham_tap.n_qbits

csvqe = csvqe_approximations_heuristic(ham, 
                                       ham_noncon, 
                                       n_qubits, 
                                       tap_gs_energy)

print('CS-VQE errors:', list(zip(range(n_qubits+1), csvqe[2])), '\n')
print('chosen order:', csvqe[3])

CS-VQE errors: [(0, 0.051774631865651344), (1, 0.0308642040442777), (2, 0.014216038483189308), (3, 0.00032650559488445197), (4, 0.00017597856924211897), (5, 1.7763568394002505e-15)] 

chosen order: [1, 2, 3, 0, 4]


# Contextual-Subspace VQE
Here we run through the basic CS-VQE functionality...

When the `cs_vqe` class is initiated it generates a set of stabilizers defined through the CS-VQE method. These stabilizers are consistent with the noncontextual ground state energy, a classical estimate of the true value that is *at least as accurate* as Hartree-Fock.

In [9]:
cs = cs_vqe(ham, noncontextual_set=terms_noncon)

#match_original = abs(cs.ngs_energy-gs_noncon[0])<1e-13
print("Noncontextual GS energy:",  cs.ngs_energy)#, ' // matches original?', match_original)
print("Symmetry generators:    ", cs.generators)
print("Clique representatives: ", cs.cliquereps)
print("Generator eigenvalues:  ", cs.nu)
print("Clique operator coeffs: ", cs.r)

Design of experiment phase, number of new doe samples = 10 .......
q0,q1,q2,q3,theta,objfnc_sum,Timestamp
1,-1,1,1,-0.26340638812180694,-13.937009379387106,2
-1,1,1,-1,0.655732329953993,-9.56198780712043,3
1,-1,-1,1,1.5028085694467315,-6.585557336939004,3
-1,-1,1,-1,-0.9657528188815192,-9.791542680212917,3
-1,1,1,1,-2.746240303571764,-14.148813074313388,4
-1,1,-1,-1,-2.1558006271347767,-10.662830123919049,4
-1,1,1,-1,1.2862172746457645,-8.218471976323732,4
1,-1,-1,1,-0.5162248153203719,-4.430273617869025,5
1,-1,1,1,2.2485619851945398,-10.771795719699105,5
-1,-1,-1,1,1.2072534026010304,-9.957258191645353,5


End of doe/resume phase, the number of evaluated configurations is: 10

Starting optimization iteration 1
q0,q1,q2,q3,theta,objfnc_sum,Timestamp
1,1,1,1,-1.9987863699010142,-6.429185710838798,833

Starting optimization iteration 2
q0,q1,q2,q3,theta,objfnc_sum,Timestamp
-1,1,1,1,-2.747198625292395,-14.149783560270555,1530

Starting optimization iteration 3
q0,q1,q2,q3,theta,objfnc_su

In [10]:
chemaccnumq = list(np.array(csvqe[2])<0.0016).index(True) #mol['chem_acc_num_q']

#exact, gs = exact_gs_energy(cs.hamiltonian)
print('Exact energy:',true_gs)
print('Noncon error:', cs.ngs_energy-true_gs)
print(f'Target_error for {chemaccnumq} qubits:', csvqe[2][chemaccnumq])# exact_gs_energy(mol['ham_reduced'][num_sim_q])[0]-exact)

Exact energy: -14.4036551080677
Noncon error: 0.051774631865670884
Target_error for 3 qubits: 0.00032650559488445197


**CS-VQE is sensitive to the choice of stabilizers we wish to enforce.**

Below, we drop stabilizer constraints iteratively, choosing that which minimizes the energy at each step.

In [11]:
stab_index_pool = list(range(len(cs.generators)))

optimal_errors = {}
for num_sim_q in range(2,cs.n_qubits):
    cs_vqe_errors = []
    for order in combinations(range(len(cs.generators)), cs.n_qubits - num_sim_q):
        order = list(order)
        ham_cs = cs.contextual_subspace_hamiltonian(stabilizer_indices=order)#list(range(cs_vqe_mol.num_qubits)),
                                                                    #projection_qubits=order)
        cs_energy, cs_vector = exact_gs_energy(ham_cs)
        cs_vqe_errors.append((cs_energy-true_gs, order))
        
    cs_vqe_errors = sorted(cs_vqe_errors, key=lambda x:x[0])
    error, stab_index_pool = cs_vqe_errors[0]
    
    optimal_errors[num_sim_q]={}
    optimal_errors[num_sim_q]['error'] = error
    optimal_errors[num_sim_q]['stab_indices'] = list(stab_index_pool)
    
for num_sim_q in optimal_errors:
    error = optimal_errors[num_sim_q]['error']
    diff_will = error-csvqe[2][num_sim_q]
    print(diff_will)
    stab_indices = optimal_errors[num_sim_q]['stab_indices']
    print(f'Performing {num_sim_q}-qubit CS-VQE, we may obtain',
          f'an absolute error of {error:.8f},\n',
          f'enforcing the stabilizers {stab_indices}, {[cs.generators[i] for i in stab_indices]}\n'
         )

0.037433368092610664
Performing 2-qubit CS-VQE, we may obtain an absolute error of 0.05164941,
 enforcing the stabilizers [1, 2, 3], ['IZIII', 'IIZII', 'IIIZI']

0.030440681243455003
Performing 3-qubit CS-VQE, we may obtain an absolute error of 0.03076719,
 enforcing the stabilizers [3, 4], ['IIIZI', 'IIIIZ']

0.013832599802814372
Performing 4-qubit CS-VQE, we may obtain an absolute error of 0.01400858,
 enforcing the stabilizers [4], ['IIIIZ']



Suppose we have access to just 3 qubits on some quantum device... then we may construct the corresponding 3-qubit CS-VQE model, obtaining the reduced Hamiltonian, Ansatz operator and noncontextual reference state.

In [ ]:
num_sim_q = 4
stab_indices = [0,1,2,3]# optimal_errors[num_sim_q]['stab_indices']
ham_cs = cs.contextual_subspace_hamiltonian(stabilizer_indices=stab_indices)
#anz_cs = cs._contextual_subspace_projection(operator=ansatz,stabilizer_indices=stab_indices)
#ngs = cs.noncontextual_ground_state(stabilizer_indices=stab_indices)

In [ ]:
print('Reduced Hamiltonian:\n', ham_cs)
print('\nReduced Ansatz:\n', anz_cs)
print('\nReference state:', ngs)

In [ ]:
plot_ground_state_amplitudes(operator=ham_cs, num_qubits=num_sim_q)#, reverse_bitstrings=True)

## Running CS-VQE

Finally, we may perform a VQE routine taking as input the reduced Hamiltonian and Ansatz defined above.

In [ ]:
import warnings; warnings.filterwarnings("ignore", category=DeprecationWarning)
import qreduce.utils.circuit_tools as circ
import qreduce.utils.circuit_execution_tools as circ_ex
from qiskit.circuit import QuantumCircuit

In [ ]:
qc = QuantumCircuit(num_sim_q)
for index, bit in enumerate(ngs):
    q_pos = num_sim_q-1-index
    if int(bit)==1:
        qc.x(q_pos)

qc = circ.circ_from_paulis(circ=qc, paulis=list(anz_cs.keys()), trot_order=2)
circ.cancel_pairs(circ=qc, hit_set={'s', 'sdg'})
circ.cancel_pairs(circ=qc, hit_set={'h', 'h'})
bounds = np.array([(a-np.pi/2, a+np.pi/2) for a in anz_cs.values()]) # optimization bounds
qc.parameter_bounds = bounds

qc.draw(output='mpl')

In [ ]:
vqe_result = circ_ex.vqe_simulation(ansatz=qc, operator=ham_cs, init_params=np.array(list(anz_cs.values())))

In [ ]:
log_errors = np.log10(np.square(np.array(vqe_result['values'])-exact))
plt.plot(log_errors, color='black', label='Optimizer output')
plt.hlines(np.log10(0.0016**2), 0, vqe_result['counts'][-1], label='Chemical accuracy', color='green')
plt.legend()